## Testing new pipeline features

In [36]:
import sys
import os

import pandas as pd

# Adds the project root to the path so it can find the 'src' contents
sys.path.append(os.path.abspath(".."))

In [37]:
from src.ingestion.pipelines.apis.ra_co import RaCoEventPipeline
from src.ingestion.base_pipeline import PipelineConfig


In [33]:
# Create config and run pipeline (live ra.co API)
config = PipelineConfig(
    source_name="ra_co",
    base_url="https://ra.co/graphql",
    batch_size=100,
    request_timeout=30,
)
pipeline = RaCoEventPipeline(config)
result = pipeline.execute()

print("📊 Fetched from live ra.co API")
print("=" * 60)

Failed to process event 1: MUSIC_AND_RHYTHM
Traceback (most recent call last):
  File "/Users/josegarcia/Documents/GitHub/event-intelligence-platform/src/ingestion/base_pipeline.py", line 375, in _process_events_batch
  File "/Users/josegarcia/Documents/GitHub/event-intelligence-platform/src/ingestion/pipelines/apis/ra_co.py", line 284, in map_to_taxonomy
    {
      
  File "/Users/josegarcia/Documents/GitHub/event-intelligence-platform/.venv/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py", line 289, in __getattr__
    raise AttributeError(item)
AttributeError: MUSIC_AND_RHYTHM
Failed to process event 2: MUSIC_AND_RHYTHM
Traceback (most recent call last):
  File "/Users/josegarcia/Documents/GitHub/event-intelligence-platform/src/ingestion/base_pipeline.py", line 375, in _process_events_batch
  File "/Users/josegarcia/Documents/GitHub/event-intelligence-platform/src/ingestion/pipelines/apis/ra_co.py", line 284, in map_to_taxonomy
    {
      
  File "/Users/jose

📊 Fetched from live ra.co API


In [34]:
# Build pandas DataFrame from pipeline results (live API)
def event_to_row(e):
    artists = e.custom_fields.get("artists") or []
    artist_names = (
        [a.get("name", "?") if isinstance(a, dict) else str(a) for a in artists[:5]]
        if isinstance(artists, list)
        else []
    )
    taxonomy = (
        "; ".join(
            f"{d.primary_category}→{d.subcategory}({d.confidence:.0%})"
            for d in (e.taxonomy_dimensions or [])
        )
        or None
    )
    return {
        "event_id": e.event_id,
        "title": e.title,
        "start_datetime": e.start_datetime,
        "end_datetime": e.end_datetime,
        "city": e.location.city,
        "country_code": e.location.country_code,
        "venue_name": e.location.venue_name,
        "artists": ", ".join(artist_names) if artist_names else None,
        "primary_category": e.primary_category,
        "taxonomy": taxonomy,
        "format": e.format,
        "is_free": e.price.is_free,
        "min_price": e.price.minimum_price,
        "max_price": e.price.maximum_price,
        "organizer": e.organizer.name,
        "source_url": e.source.source_url,
        "data_quality_score": e.data_quality_score,
    }

COLS = [
    "event_id", "title", "start_datetime", "end_datetime", "city", "country_code",
    "venue_name", "artists", "primary_category", "taxonomy", "format",
    "is_free", "min_price", "max_price", "organizer", "source_url", "data_quality_score",
]
rows = [event_to_row(e) for e in result.events]
df = pd.DataFrame(rows, columns=COLS) if rows else pd.DataFrame(columns=COLS)
print(f"✅ Total events: {len(df)}")
df

✅ Total events: 0


""


In [ ]:
print("=" * 60)
print(f"Pipeline status: {result.status}")
print(f"Duration: {result.duration_seconds:.2f}s")
print(f"Success rate: {result.success_rate:.1f}%")
print("=" * 60)
